In [1]:
import os
import sys
import cv2

import random

import numpy as np

from tqdm import tqdm
import pickle

from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense

from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


In [2]:
os.listdir('data')

['train', 'test', 'sample_submission.csv']

In [3]:
len(os.listdir('data/train'))

25000

In [4]:
train_images = os.listdir('data/train')
test_images = os.listdir('data/test')

print(len(train_images))
print(len(test_images))

25000
12500


In [5]:
train_images[:10]

['cat.4213.jpg',
 'cat.7203.jpg',
 'dog.8250.jpg',
 'dog.7907.jpg',
 'dog.2318.jpg',
 'cat.6480.jpg',
 'dog.7973.jpg',
 'dog.2225.jpg',
 'dog.611.jpg',
 'dog.5955.jpg']

In [6]:
test_images[:10]

['1818.jpg',
 '374.jpg',
 '1681.jpg',
 '2880.jpg',
 '2001.jpg',
 '11288.jpg',
 '4282.jpg',
 '7510.jpg',
 '11638.jpg',
 '8288.jpg']

In [8]:
test_images_data = []

for image in tqdm(test_images):
    image_data = cv2.imread('data/test/' + image)
    
    #Convert to GrayScale
    #gray = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
    
    #convert color from BGR to RGB
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    
    image_data = cv2.resize(image_data, (64, 64))
    
    #turn to only borders
    #image_data = cv2.Canny(image_data, 150, 150)
    
    test_images_data.append(image_data)

100%|██████████| 12500/12500 [00:49<00:00, 251.23it/s]


In [9]:
test_images_data = np.array(test_images_data)

In [10]:
train_images_data = []
train_images_labels = []

random.shuffle(train_images)

for image in tqdm(train_images):
    image_data = cv2.imread('data/train/' + image)
    
    #Convert to GrayScale
    #gray = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
    
    #convert color from BGR to RGB
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    
    image_data = cv2.resize(image_data, (64, 64))
    
    #turn to only borders
    #image_data = cv2.Canny(image_data, 150, 150)
    
    train_images_data.append(image_data)
    train_images_data.append(image_data[:, ::-1]) #flipped image
    
    
    if image.startswith('cat'):
        train_images_labels.append(0)
        train_images_labels.append(0)
        #train_images_labels.append([0, 1])
    else:
        train_images_labels.append(1)
        train_images_labels.append(1)
        #train_images_labels.append([1, 0])

100%|██████████| 25000/25000 [01:39<00:00, 251.51it/s]


In [11]:
train_images_data = np.array(train_images_data)
train_images_labels = np.array(train_images_labels)

train_images_data = train_images_data.reshape([-1, 64, 64, 3])

In [21]:
pickle.dump(train_images_data, open('model_data/colored_train_images_data.pck', 'wb'))

In [22]:
pickle.dump(test_images_data, open('model_data/colored_test_images_data.pck', 'wb'))

In [23]:
pickle.dump(train_images_labels, open('model_data/colored_train_images_labels.pck', 'wb'))

In [20]:
model_name = 'colored_model_epochs_100_conv_3_by_3' #

In [24]:
model = Sequential() 
model.add(Conv2D(64, (3, 3), input_shape=(64, 64, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
  
model.add(Conv2D(32, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
  
model.add(Conv2D(16, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
  
model.add(Flatten()) 
model.add(Dense(50)) 
model.add(Activation('relu')) 
#model.add(Dropout(0.5)) 
model.add(Dense(1)) 
model.add(Activation('sigmoid')) 

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']) 

tensorboard = TensorBoard(log_dir="logs/{}".format(model_name))
early_stop = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint(filepath='model_data/' + model_name + '_checkpoint.h5', monitor='val_loss', 
                             save_best_only=True)

model.fit(train_images_data, train_images_labels, epochs=100, validation_split=0.2, 
          callbacks=[tensorboard, early_stop, checkpoint])

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 824s 21ms/step - loss: 0.8781 - acc: 0.5254 - val_loss: 0.6883 - val_acc: 0.5446
Epoch 2/100
40000/40000 [==============================] - 776s 19ms/step - loss: 0.6868 - acc: 0.5354 - val_loss: 0.6869 - val_acc: 0.5446
Epoch 3/100
40000/40000 [==============================] - 780s 19ms/step - loss: 0.6828 - acc: 0.5485 - val_loss: 0.6814 - val_acc: 0.5646
Epoch 4/100
40000/40000 [==============================] - 751s 19ms/step - loss: 0.6793 - acc: 0.5537 - val_loss: 0.6758 - val_acc: 0.5752
Epoch 5/100
40000/40000 [==============================] - 674s 17ms/step - loss: 0.6724 - acc: 0.5765 - val_loss: 0.6687 - val_acc: 0.5917
Epoch 6/100
40000/40000 [==============================] - 673s 17ms/step - loss: 0.6629 - acc: 0.5987 - val_loss: 0.6685 - val_acc: 0.5972
Epoch 7/100
40000/40000 [==============================] - 671s 17ms/step - loss: 0.6352 - acc: 0.6378 - val_l

In [17]:
#save the data as pickle files
#save the model
#run on test example images, about 10 to see classification

In [25]:
model.save('models/' + model_name + '.h5')